In [49]:
import numpy as np
from astropy.io import fits
from astropy.coordinates import SkyCoord
from astropy import units as u

# read Zakamska catalog
hdu = fits.open("Zakamska2003.fit")
data = hdu[1].data
catalog1 = SkyCoord(ra=data['_RA']*u.deg, dec=data['_DE']*u.deg)
hdu.close()

# same for Reyes
hdu = fits.open("Reyes2008.fit")
data = hdu[1].data
catalog2 = SkyCoord(ra=data['RAJ2000']*u.deg, dec=data['DEJ2000']*u.deg)
hdu.close()

# for Mullaney we have to select agn type 2
hdu = fits.open("Mullaney2013.fits")
data = hdu[1].data
sel = data['AGN_TYPE'] == 2
catalog3 = SkyCoord(ra=data['RA'][sel]*u.deg, dec=data['DEC'][sel]*u.deg)
hdu.close()

# for Yuan, we have to pick unique ones
hdu = fits.open("Yuan2016.fits")
data = hdu[1].data
sel = data['unique'] == 'unique'
catalog4 = SkyCoord(ra=data['ra'][sel]*u.deg, dec=data['dec'][sel]*u.deg)
hdu.close()

In [50]:
def xor_catalog(catalog1, catalog2, sep=1*u.arcsec):
    idx1, idx2, _, _ = catalog2.search_around_sky(catalog1, sep)
    notin2 = np.in1d(np.arange(len(catalog2)), idx2, invert=True)
    ra = np.concatenate((np.array(catalog1.ra), np.array(catalog2.ra)[notin2]))
    dec = np.concatenate((np.array(catalog1.dec), np.array(catalog2.dec)[notin2]))
    return SkyCoord(ra=ra*u.deg, dec=dec*u.deg)

In [59]:
# matching catalogs in the order of Sun et al. (2018) Table 1
catalog42 = xor_catalog(catalog4, catalog2, sep=3*u.arcsec)
catalog421 = xor_catalog(catalog42, catalog1, sep=3*u.arcsec)
catalog4213 = xor_catalog(catalog421, catalog3, sep=2*u.arcsec)

# however, these numbers don't agree with Table 1
# additional selections need to have happened
# e.g. Ai-Lei rejected 2 AGNs from Mullaney because they were apparently Type 1 
len(catalog4213), len(catalog421), len(catalog42), len(catalog4), len(catalog2)

(17059, 3746, 3601, 2758, 887)

In [65]:
# save as fits
from astropy.table import Table
t = Table((catalog4213.ra, catalog4213.dec), names=['RA','DEC'])
t.write('parent.fits')